In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"), 
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

2024-08-27 21:30:14.573552: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-27 21:30:14.602737: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 21:30:15.047490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("/home/eiji/project/g/preprocessed_data/Fridge_LE.csv")
data

,hour,minute,second,fridge_temperature,temp_condition_encoded,label
0,12,36,52,13.10,1,0
1,12,36,53,8.65,1,0
2,12,36,54,2.00,0,0
3,12,36,55,4.80,0,0
4,12,36,56,10.70,1,0
...,...,...,...,...,...,...
587071,12,41,17,4.00,0,0
587072,12,41,17,6.05,1,0
587073,12,41,17,12.80,1,0
587074,12,41,18,6.50,1,0


In [3]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2024-08-27 21:30:15.821567: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-27 21:30:15.844642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-27 21:30:15.844785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
X = data[['hour', 'minute', 'second', 'fridge_temperature', 'temp_condition_encoded']].values
y = data['label'].values

In [5]:
# トレーニングデータとテストデータに分割
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# テストデータをさらに検証データとテストデータに分割
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [6]:
def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size + 1):
        sequences.append(data[i:i + window_size])
    return np.array(sequences)

window_size = 30

X_train_seq = create_sequences(X_train, window_size)
X_val_seq = create_sequences(X_val, window_size)
X_test_seq = create_sequences(X_test, window_size)

# ターゲット（y）も対応する最後のウィンドウ位置に合わせる
y_train_seq = y_train[window_size - 1:]
y_val_seq = y_val[window_size - 1:]
y_test_seq = y_test[window_size - 1:]


In [7]:
# Transformerモデルの構築
embed_dim = X_train.shape[1]  # Input feature size
num_heads = 8
ff_dim = 256  # Hidden layer size in feed forward network inside transformer

# モデルの構築
input_layer = tf.keras.layers.Input(shape=(window_size, X_train.shape[1]))
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(input_layer)
x = Flatten()(x)  # Flatten the output to match the target shape
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 30, 5)          │         3,766 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,917 (15.30 KB)

 Trainable params: 3,917 (15.30 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# モデルの訓練
# モデルの訓練
# history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)
# history = model.fit(X_train.reshape(-1, 1, X_train.shape[1]), y_train, epochs=10, batch_size=64, validation_data=(X_val.reshape(-1, 1, X_val.shape[1]), y_val), verbose=1, shuffle=False)

history = model.fit(
    X_train_seq, y_train_seq,
    epochs=10,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    verbose=1,
    shuffle=False
)


Epoch 1/10


I0000 00:00:1724761829.612393   37220 service.cc:145] XLA service 0x719354012020 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724761829.612412   37220 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-08-27 21:30:29.648368: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  21/6421 ━━━━━━━━━━━━━━━━━━━━ 55s 9ms/step - accuracy: 0.8426 - loss: 0.4628 

I0000 00:00:1724761830.328884   37220 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6421/6421 ━━━━━━━━━━━━━━━━━━━━ 97s 15ms/step - accuracy: 0.8531 - loss: 0.4023 - val_accuracy: 0.8531 - val_loss: 0.3619
Epoch 2/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 120s 19ms/step - accuracy: 0.8530 - loss: 0.3633 - val_accuracy: 0.8531 - val_loss: 0.3561
Epoch 3/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 98s 15ms/step - accuracy: 0.8531 - loss: 0.3577 - val_accuracy: 0.8531 - val_loss: 0.3491
Epoch 4/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 83s 13ms/step - accuracy: 0.8529 - loss: 0.3529 - val_accuracy: 0.8531 - val_loss: 0.3445
Epoch 5/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 89s 14ms/step - accuracy: 0.8529 - loss: 0.3499 - val_accuracy: 0.8531 - val_loss: 0.3466
Epoch 6/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - accuracy: 0.8529 - loss: 0.3482 - val_accuracy: 0.8531 - val_loss: 0.3462
Epoch 7/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.8529 - loss: 0.3473 - val_accuracy: 0.8531 - val_loss: 0.3434
Epoch 8/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 77s 12ms/step - accuracy: 0.8530 - loss: 0.3

In [9]:
# モデルの評価
# loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
loss, accuracy = model.evaluate(X_test.reshape(-1, 1, X_test.shape[1]), y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

2024-08-27 21:49:14.032137: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:580 : INVALID_ARGUMENT: only one input size may be -1, not both 0 and 1

Stack trace for op definition: 
File "home/eiji/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
File "home/eiji/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 701, in start
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
File "home/eiji/miniconda3/envs/tf/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
File "home/eiji/miniconda3/envs/tf/lib/python3.9/asyncio/bas

InvalidArgumentError: Graph execution error:

Detected at node functional_2_1/flatten_1/Reshape defined at (most recent call last):
<stack traces unavailable>
only one input size may be -1, not both 0 and 1

Stack trace for op definition: 
File "home/eiji/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
File "home/eiji/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 701, in start
File "home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
File "home/eiji/miniconda3/envs/tf/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
File "home/eiji/miniconda3/envs/t

In [10]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 予測の実行
y_pred = model.predict(X_test.reshape(-1, 1, X_test.shape[1]))
y_pred_classes = (y_pred > 0.5).astype(int)  # 閾値0.5でクラスに変換


# 混同行列の計算
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# 混同行列の表示
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 精度、適合率、再現率、F1スコアのレポート
print(classification_report(y_test, y_pred_classes))


ValueError: Input 0 of layer "functional_2" is incompatible with the layer: expected shape=(None, 30, 5), found shape=(32, 1, 5)